In [1]:
# !pip install kaggle
# !pip install numpy pandas numerize lets_plot
# !pip install python-dotenv
# !pip install plotnine
# remove this chunk and add instruction in 'README' file

In [2]:
import kaggle
import csv
import os
import json
import numpy as np
import pandas as pd

from numerize import numerize as nz

from lets_plot import *
from lets_plot.mapping import *
LetsPlot.setup_html()
from dotenv import dotenv_values

# Part I: Data Collection

## Setting up credentials

- We aimis to collect raw crime data from Kaggle.
- To collect datasets, what we need to do first is to obtain API credentials from Kaggle. 

In [3]:
config = dotenv_values("../.env")

kaggle_username = config["KAGGLE_USERNAME"]
kaggle_key = config["KAGGLE_KEY"]

# Configurer les variables d'environnement pour Kaggle
os.environ["KAGGLE_USERNAME"] = kaggle_username
os.environ["KAGGLE_KEY"] = kaggle_key

- Now we are allowed to use the Kaggle API to interact with Kaggle datasets. We extracted crime data for both Baltimore and Vancouver as our raw data, and store them separately in two dataframes named `df_baltimore` and `df_vancouver`.

In [4]:
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# Save the credentials to the kaggle.json file
api_token = {"username": kaggle_username, "key": kaggle_key}
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as file:
    json.dump(api_token, file)

# Set permissions for the kaggle.json file
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

# Define the download path
download_path = r'../data/raw/'


# Create the download directory if it does not exist
os.makedirs(download_path, exist_ok=True)

# Download the dataset to the specified path
kaggle.api.dataset_download_files('sohier/crime-in-baltimore', path=download_path, unzip=True)

# Load the dataset into a pandas DataFrame
# Assuming the dataset has a CSV file, you may need to adjust the file name based on the actual dataset
csv_file = [file for file in os.listdir(download_path) if file.endswith('.csv')][0]
csv_path = os.path.join(download_path, csv_file)
df_baltimore = pd.read_csv(csv_path)
df_baltimore

Dataset URL: https://www.kaggle.com/datasets/sohier/crime-in-baltimore


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,District,Neighborhood,Longitude,Latitude,Location 1,Premise,Total Incidents
0,09/02/2017,23:30:00,3JK,4200 AUDREY AVE,ROBBERY - RESIDENCE,I,KNIFE,913.0,SOUTHERN,Brooklyn,-76.60541,39.22951,"(39.2295100000, -76.6054100000)",ROW/TOWNHO,1
1,09/02/2017,23:00:00,7A,800 NEWINGTON AVE,AUTO THEFT,O,NaN,133.0,CENTRAL,Reservoir Hill,-76.63217,39.31360,"(39.3136000000, -76.6321700000)",STREET,1
2,09/02/2017,22:53:00,9S,600 RADNOR AV,SHOOTING,Outside,FIREARM,524.0,NORTHERN,Winston-Govans,-76.60697,39.34768,"(39.3476800000, -76.6069700000)",Street,1
3,09/02/2017,22:50:00,4C,1800 RAMSAY ST,AGG. ASSAULT,I,OTHER,934.0,SOUTHERN,Carrollton Ridge,-76.64526,39.28315,"(39.2831500000, -76.6452600000)",ROW/TOWNHO,1
4,09/02/2017,22:31:00,4E,100 LIGHT ST,COMMON ASSAULT,O,HANDS,113.0,CENTRAL,Downtown West,-76.61365,39.28756,"(39.2875600000, -76.6136500000)",STREET,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276524,01/01/2012,00:00:00,6J,1400 JOH AVE,LARCENY,I,NaN,832.0,SOUTHWESTERN,Violetville,-76.67195,39.26132,"(39.2613200000, -76.6719500000)",OTHER - IN,1
276525,01/01/2012,00:00:00,6J,5500 SINCLAIR LN,LARCENY,O,NaN,444.0,NORTHEASTERN,Frankford,-76.53829,39.32493,"(39.3249300000, -76.5382900000)",OTHER - OU,1
276526,01/01/2012,00:00:00,6E,400 N PATTERSON PK AV,LARCENY,O,NaN,321.0,EASTERN,CARE,-76.58497,39.29573,"(39.2957300000, -76.5849700000)",STREET,1
276527,01/01/2012,00:00:00,5A,5800 LILLYAN AV,BURGLARY,I,NaN,425.0,NORTHEASTERN,Glenham-Belhar,-76.54578,39.34701,"(39.3470100000, -76.5457800000)",APT. LOCKE,1


In [5]:
download_path = r'../data/raw/'

# Create the download directory if it does not exist
os.makedirs(download_path, exist_ok=True)

# Download the dataset to the specified path
kaggle.api.dataset_download_files('wosaku/crime-in-vancouver', path=download_path, unzip=True)

# Load the dataset into a pandas DataFrame
# Assuming the dataset has a CSV file, you may need to adjust the file name based on the actual dataset
csv_file = [file for file in os.listdir(download_path) if file.endswith('.csv')][0]
csv_path = os.path.join(download_path, csv_file)
df_vancouver = pd.read_csv('../data/raw/crime.csv')
df_vancouver.head()

Dataset URL: https://www.kaggle.com/datasets/wosaku/crime-in-vancouver


,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y,Latitude,Longitude
0,Other Theft,2003,5,12,16.0,15.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763
1,Other Theft,2003,5,7,15.0,20.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763
2,Other Theft,2003,4,23,16.0,40.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763
3,Other Theft,2003,4,20,11.0,15.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763
4,Other Theft,2003,4,12,17.0,45.0,9XX TERMINAL AVE,Strathcona,493906.5,5457452.47,49.269802,-123.083763


In [6]:
!pip freeze > requirement.txt